## Problem 2
## 2(a) & (b)

In [27]:
# Import packages
import numpy as np
import pandas as pd
import scipy.stats as sts
import os
import matplotlib.pyplot as plt
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')
from scipy import special
from scipy.stats import norm

In [28]:
# Load data
data = np.loadtxt('data/MacroSeries.txt', delimiter=",")
beta = 0.99
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]
print(c.mean())
print(k.mean())
print(w.mean())
print(r.mean())

10520847.8208
7472544.55731
10445210.0925
1.01694730195


In [29]:
# Define functions
def get_z1(w, k, alpha):

    z = np.log(w) - np.log(1 - alpha) - alpha * np.log(k)
    
    return z

def get_z2(r, k, alpha):
    
    z = np.log(r) - np.log(alpha) - (alpha - 1) * np.log(k)
    
    return z

def logpdf(z, rho, mu, sigma):
    z[0] = mu
    mean = rho * z[:-1] + (1 - rho) * mu
    logpdf_vals = norm.logpdf(z, np.hstack([mu, mean]), sigma)
    
    return logpdf_vals

def crit1(params, *args):
   
    alpha, rho, mu, sigma = params
    w, k  = args
    neglogpdf_vals = -(logpdf(get_z1(w, k, alpha), rho, mu, sigma).sum())
    
    return neglogpdf_vals

def crit2(params, *args):
   
    alpha, rho, mu, sigma = params
    r, k  = args
    neglogpdf_vals = -(logpdf(get_z2(r, k, alpha), rho, mu, sigma).sum())
    
    return neglogpdf_vals

In [30]:
alpha_init = 0.5
rho_init = 0.7
mu_init = 10
sigma_init = 0.1
params_init = (alpha_init, rho_init, mu_init, sigma_init)

args1 = (w, k)
args2 = (r, k)
bnds = ((1e-6, 1-1e-6), (-1+1e-6, 1-1e-6),(1e-6, None), (1e-6, None))

results1 = opt.minimize(crit1, params_init, args=(args1), method ='L-BFGS-B', bounds=bnds)
alpha1, rho1, mu1, sigma1 = results1.x
print(results1)
# Inverse Hessian
results1.hess_inv.todense()

      fun: -96.831236726610783
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00439826, -0.00099334,  0.08546124, -0.00680274])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 175
      nit: 18
   status: 0
  success: True
        x: array([ 0.42489926,  0.74681636,  9.99267974,  0.09188164])


array([[  5.50347212e-03,   1.77295994e-02,  -6.82994418e-02,
          5.40152352e-04],
       [  1.77295994e-02,   6.14276700e-01,   5.61545298e-02,
          1.59405566e-02],
       [ -6.82994418e-02,   5.61545298e-02,   9.95797306e-01,
          2.70918677e-04],
       [  5.40152352e-04,   1.59405566e-02,   2.70918677e-04,
          4.83939365e-04]])

In [31]:
results2 = opt.minimize(crit2, params_init, args=(args2), method ='L-BFGS-B', bounds=bnds)
alpha2, rho2, mu2, sigma2 = results2.x
print(results2)
# Inverse Hessian
results2.hess_inv.todense()

      fun: -96.862599913884907
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.05567102, -0.00134861,  0.00307381,  0.03353477])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 325
      nit: 44
   status: 0
  success: True
        x: array([ 0.48640087,  0.70182713,  8.86076137,  0.09185453])


array([[  5.54808616e+01,  -1.15373987e+02,  -1.00639304e+03,
         -3.77172198e-01],
       [ -1.15373987e+02,   2.41943157e+02,   2.09299559e+03,
          8.82038069e-01],
       [ -1.00639304e+03,   2.09299559e+03,   1.82554466e+04,
          6.84944269e+00],
       [ -3.77172198e-01,   8.82038068e-01,   6.84944269e+00,
          7.53550783e-03]])

## 2(c)

In [33]:
z_star = get_z2(1, 7500000, alpha2)
mean = rho2 * 10 + (1 - rho2) * mu2
print('Probability is: ' + str(1-norm.cdf(z_star, mean, sigma2)))

Probability is: 1.0
